# Finetune

Based on: https://huggingface.co/docs/transformers/tasks/translation

In [1]:
import sys, os
import json
import torch
import random
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TranslationPipeline
from datasets import load_dataset, Dataset

In [2]:
source_langs = set(["akkts", "suxts"])

target_langs = set(["en", "it", "es", "fr", "de"])

base_model_id = "t5-small"

model_max_length = 128
batch_size = 256

In [3]:
model_id = f"{base_model_id}-{'-'.join(source_langs)}"
model_id

't5-small-akkts-suxts'

In [4]:
has_cuda = torch.cuda.is_available()

In [5]:
device = torch.cuda.device(0) if has_cuda else "cpu"
device

## Load Training Data

In [6]:
lang_full = {
    "akkts": "Akkadian",
    "suxts": "Sumerian",
    "en": "English",
    "it": "Italian",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
}

In [7]:
def get_prefix(src_lang, tgt_lang):
    s = lang_full[src_lang]
    t = lang_full[tgt_lang]
    return f"translate {s} to {t}: "
    
get_prefix("suxts", "es")

'translate Sumerian to Spanish: '

In [8]:
translations = load_dataset("json", data_files="../data/translations.jsonl")
translations

Using custom data configuration default-dc8b07d8fd701d7f
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-dc8b07d8fd701d7f/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['p', 'a', 'l', 'sux', 'en', 'akk', 'akkts', 'de', 'suxts', 'fr', 'elx', 'es', 'it'],
        num_rows: 74584
    })
})

In [9]:
sourceandtargets = []
for s in source_langs:
    print("Preparing", s)
    for t in tqdm(target_langs):
        prefix = get_prefix(s, t)
        for line in translations["train"]:
            ls = line[s]
            lt = line[t]
            if ls is not None and lt is not None:
                if lt[-1] == "." or lt[-1] == "!" or lt[-1] == ";" or lt[-1] == ",":
                    lt = lt[:-1]
                sourceandtargets.append((prefix + ls, lt))
                
random.shuffle(sourceandtargets)
translations = Dataset.from_dict({"source": [x[0] for x in sourceandtargets], "target": [x[1] for x in sourceandtargets]})
translations

Preparing akkts


  0%|          | 0/5 [00:00<?, ?it/s]

Preparing suxts


  0%|          | 0/5 [00:00<?, ?it/s]

Dataset({
    features: ['source', 'target'],
    num_rows: 7427
})

In [10]:
translations = translations.train_test_split(test_size=0.1)
translations

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 6684
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 743
    })
})

In [11]:
tests = translations["test"]
tests

Dataset({
    features: ['source', 'target'],
    num_rows: 743
})

## Tokenize the Data

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, model_max_length=model_max_length)

In [32]:
print("pad", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos", tokenizer.eos_token, tokenizer.eos_token_id)
print("unk", tokenizer.unk_token, tokenizer.unk_token_id)

pad <pad> 0
eos </s> 1
unk <unk> 2


In [35]:
ccc = 0

def preprocess_function(examples):
    global ccc
#     print(examples)
    inputs = [example for example in examples["source"]]
    targets = [example for example in examples["target"]]
    model_inputs = tokenizer(inputs, max_length=model_max_length, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    ccc += 1
    if ccc == 1:
        print(model_inputs["input_ids"][0])
        print(model_inputs["labels"][0])

    return model_inputs

tokenized_translations = translations.map(preprocess_function, batched=True)
tokenized_translations

  0%|          | 0/7 [00:00<?, ?ba/s]

[13959, 4823, 1258, 8603, 12, 1566, 10, 3, 1258, 7, 9, 102, 3, 173, 1824, 10443, 1]
[8, 4294, 3, 88, 808, 1]


  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6684
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 743
    })
})

In [14]:
tokenized_translations["train"].remove_columns(["source", "target"])
tokenized_translations["test"].remove_columns(["source", "target"])

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 743
})

In [15]:
source_max_length = max([len(x["input_ids"]) for x in tokenized_translations["train"]])
target_max_length = max([len(x["labels"]) for x in tokenized_translations["train"]])
source_max_length, target_max_length

(59, 44)

In [16]:
tokenized_translations["train"][0]["labels"][:10]

[8, 4294, 3, 88, 808, 1]

## Load the Model

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)

In [18]:
model.config

T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translation_

## Train

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
data_collator

DataCollatorForSeq2Seq(tokenizer=PreTrainedTokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_len=128, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<ext

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../results/{model_id}",
    evaluation_strategy="epoch",
    learning_rate=2*2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=400,
    fp16=has_cuda,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_translations["train"],
    eval_dataset=tokenized_translations["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



Using amp half precision backend


In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/fak/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6684
  Num Epochs = 400
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 10800
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it man

Epoch,Training Loss,Validation Loss
1,No log,4.482427
2,No log,4.182754
3,No log,4.027725
4,No log,3.914652
5,No log,3.829299
6,No log,3.765722
7,No log,3.707196
8,No log,3.654407
9,No log,3.609337
10,No log,3.569222


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examp

***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can

***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can

...

***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can

***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can

***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can

***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 743
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: source, target. If source, target are not expected by `T5ForConditionalGeneration.forward`,  you can

TrainOutput(global_step=10800, training_loss=2.0012527324535228, metrics={'train_runtime': 1758.0655, 'train_samples_per_second': 1520.762, 'train_steps_per_second': 6.143, 'total_flos': 3.3361432626069504e+16, 'train_loss': 2.0012527324535228, 'epoch': 400.0})

## Sample

In [22]:
pipeline = TranslationPipeline(model=model.to("cpu"), tokenizer=tokenizer, max_length=model_max_length)

In [23]:
pipeline

In [24]:
pipeline("translate English to French: hello my name is Frank")

[{'translation_text': 'Bonjour, mon nom est Frank'}]

In [25]:
source_test = translations["test"][0]["source"]
target_test = translations["test"][0]["target"]
print(source_test)
print("-"*80)
print(target_test)

translate Akkadian to English: šina šiqil kaspam
--------------------------------------------------------------------------------
2 shekels of silver


In [26]:
def translate(text):
    return pipeline(prefix + source_test)

translate("ina ebūrim")

[{'translation_text': '1 shekel of silver'}]

In [27]:
def sample(num_samples=100):
    for i in range(min(num_samples, tests.num_rows)):
        t = tests[i]
    #     print(t)
        src = t["source"]
        tgt = t["target"]
        query = src
        pred = pipeline(query)[0]["translation_text"]
        print("-"*48)
        print("QUERY ", query)
        print("TARGET", tgt)
        print("PRED  ", pred)
    #     break
    
sample()

------------------------------------------------
QUERY  translate Akkadian to English: šina šiqil kaspam
TARGET 2 shekels of silver
PRED   2 shekels of silver
------------------------------------------------
QUERY  translate Akkadian to English: ītelli
TARGET shall forfeit
PRED   he shall depart
------------------------------------------------
QUERY  translate Akkadian to English: uštākil
TARGET feeds
PRED   has caused to be seized
------------------------------------------------
QUERY  translate Akkadian to English: ina balum bēl še’im
TARGET without (obtaining permission from) the owner of the barley
PRED   without the consent of the owner of the barley
------------------------------------------------
QUERY  translate Akkadian to English: kaspam mišil šīmiša
TARGET silver of half of its value
PRED   silver of half of its value
------------------------------------------------
QUERY  translate Akkadian to English: e’iltum
TARGET an obligation
PRED   the e’iltum
------------------------

------------------------------------------------
QUERY  translate Akkadian to English: ina ḫiṭāni gabbi lū napāli lū
TARGET In all punishments, whether gouging or
PRED   at the end of the night, nothing was appropriate
------------------------------------------------
QUERY  translate Akkadian to English: narêšu
TARGET his stela
PRED   his stela
------------------------------------------------
QUERY  translate Akkadian to English: ša ṭēmišu
TARGET at his disposal
PRED   of his day
------------------------------------------------
QUERY  translate Akkadian to English: ultaṣbissi u kî aššat
TARGET causes her to take on and that the wife of
PRED   has seized her and as his wife
------------------------------------------------
QUERY  translate Akkadian to English: riksātušu laššu
TARGET his contract there is none
PRED   his contract has none
------------------------------------------------
QUERY  translate Akkadian to English: adi šūnu ina gimiršunu kimin
TARGET while they in their entirety 